# `simple_dmrg` demonstration

See the README.md for documentation on how to install `simple_dmrg`

To run this notebook, run the following command in a terminal:
`jupyter notebook`

Then, open this file in the browser window that opens.

More information Jupyter notebooks can be found here: https://www.geeksforgeeks.org/getting-started-with-jupyter-notebook-python/

In [1]:
# Imports
import numpy as np
import simple_dmrg.dmrg_functions as dmrg
import simple_dmrg.mps_functions as mps_mpo
import simple_dmrg.mpo_operations as mpo_ops
import simple_dmrg.mpo_construction as make_mpo

In [2]:
# Implement the Hamiltonian as a matrix product operator
# The Hamiltonian is simple_dmrg.mpo_constructionwhere c_i^† is the fermionic creation operator for site i,
# and h_ij is the one-body tensor element between sites i and j.

# Parameters
num_sites = 4

# # Define the one-body tensor
# # Just use a simple diagonal matrix for illustrative purposes
# one_body_tensor = np.zeros(shape=(num_sites,num_sites))
# one_body_tensor[0,0] = 1
# one_body_tensor[1,1] = -3
# one_body_tensor[2,2] = -1
# one_body_tensor[3,3] = 1
# one_body_mpo = make_mpo.make_one_body_mpo(one_body_tensor=one_body_tensor, num_sites=one_body_tensor.shape[0])

# Define the two-body tensor
# Just use a simple diagonal tensor for illustrative purposes
two_body_tensor = np.zeros(shape=(num_sites,num_sites,num_sites,num_sites))
two_body_tensor[0,0,0,0] = 1
two_body_tensor[1,1,1,1] = -3
two_body_tensor[2,2,2,2] = -1
two_body_tensor[3,3,3,3] = 1
two_body_mpo = make_mpo.make_two_body_mpo(two_body_tensor=two_body_tensor, num_sites=two_body_tensor.shape[0])


In [3]:
# N_e is the number of electrons, and μ > 0 is the penalty.

penalty = 1000 # μ 
num_particles = 3 # N_e

# Add a number penalty term to the Hamiltonian
# ∑_i n_i
particle_number_mpo = make_mpo.make_particle_number_mpo(num_sites=4) 

# -∑_i n_i
negative_part_num_mpo = mpo_ops.mpo_mult_by_scalar(
    mpo=particle_number_mpo, scalar=-1, deposit_site=0
)
# Identity operator
identity_mpo = make_mpo.make_identity_mpo(num_sites=4, num_physical_dims=2)

#N_e
const_identity_mpo = mpo_ops.mpo_mult_by_scalar(
    mpo=identity_mpo, scalar=num_particles, deposit_site=0
)

#N_e - ∑_i n_i
id_min_part_num_mpo = mpo_ops.add_mpos(mpo1=const_identity_mpo, mpo2=negative_part_num_mpo)
id_min_part_num_mpo_copy  = id_min_part_num_mpo.copy()

# (N_e - ∑_i n_i)^2
id_min_part_num_sq_mpo = mpo_ops.multiply_mpos(mpo1=id_min_part_num_mpo, mpo2=id_min_part_num_mpo_copy)

# μ  (N_e - ∑_i n_i)^2
id_min_part_num_sq_mpo = mpo_ops.mpo_mult_by_scalar(
    mpo=id_min_part_num_sq_mpo, scalar=penalty, deposit_site=None
)

In [4]:
# Add the penalty term to the Hamiltonian
two_body_mpo_with_penalty = mpo_ops.add_mpos(mpo1=two_body_mpo, mpo2=id_min_part_num_sq_mpo)

In [5]:
# Check that the Hamiltonian is Hermitian

# Convert the Hamiltonian MPO to a dense matrix
hamiltonian_matrix  = mpo_ops.mpo_to_dense_matrix(mpo=two_body_mpo_with_penalty)

print("hamiltonian_matrix is Hermitian:",np.allclose(hamiltonian_matrix, hamiltonian_matrix.T.conj()))
print("Max absolute difference between hamiltonian_matrix and hamiltonian_matrix.T.conj():",np.max(np.abs(hamiltonian_matrix - hamiltonian_matrix.T.conj())))


hamiltonian_matrix is Hermitian: True
Max absolute difference between hamiltonian_matrix and hamiltonian_matrix.T.conj(): 0.0


In [6]:
# Run DMRG to find the ground state of the Hamiltonian

# Parameters
bond_dimension = 10

optimized_mps = dmrg.drmg_main(
    mpo=two_body_mpo_with_penalty,
    num_sites = num_sites,
    physical_dimension = 2, # 2 for one possible Fermion per site (aka spin-orbital)
    bond_dimension= bond_dimension,
    seed = 0, # Random seed for the initial state
    num_sweeps = 3, # Number of DMRG sweeps. Each sweep includes a left-to-right and right-to-left sweep
    verbosity = 1, # 0: No output, 1: Basic output, 2: Detailed output for debugging
)

Random MPS generated (non-normalized).
MPS transformed to right-canonical form.
Calculating initial L and R tensors...
Initial L and R tensors calculated.
---------------------
Sweep  0 , left-to-right , site  0
Sweep  0 , left-to-right , site  1
Sweep  0 , left-to-right , site  2
Sweep  0 , left-to-right , site  3
---------------------
Sweep  0 , right-to-left , site  3
Sweep  0 , right-to-left , site  2
Sweep  0 , right-to-left , site  1
Sweep  0 , right-to-left , site  0
---------------------
Sweep  1 , left-to-right , site  0
Sweep  1 , left-to-right , site  1
Sweep  1 , left-to-right , site  2
Sweep  1 , left-to-right , site  3
---------------------
Sweep  1 , right-to-left , site  3
Sweep  1 , right-to-left , site  2
Sweep  1 , right-to-left , site  1
Sweep  1 , right-to-left , site  0
---------------------
Sweep  2 , left-to-right , site  0
Sweep  2 , left-to-right , site  1
Sweep  2 , left-to-right , site  2
Sweep  2 , left-to-right , site  3
---------------------
Sweep  2 , ri

In [7]:
# Calculate the energy of the optimized_mps, the ground state energy is -3.0

gs_energy = mpo_ops.mpo_general_expectation(mps_bra=optimized_mps,mpo=two_body_mpo_with_penalty,mps_ket=optimized_mps)
print("Ground state energy:",gs_energy)

Ground state energy: (-2.9999999999965983+0j)


In [8]:
# Calculate the particle number of optimized_mps, which should be 3

gs_num_particles = mpo_ops.mpo_general_expectation(mps_bra=optimized_mps,mpo=particle_number_mpo,mps_ket=optimized_mps)
print("Ground state particle number:",gs_num_particles)

Ground state particle number: (3.0000000000000018+0j)
